In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor


In [2]:
import pandas as pd
from utils.preprocess import treat_dataset_pandas_init, build_preprocessing_pipeline,INIT_NUMERICAL_COLS, numerical_features

In [3]:
train_data = pd.read_csv("playground-series-s4e12/train.csv")
test_data = pd.read_csv("playground-series-s4e12/test.csv")

treated_dataset = treat_dataset_pandas_init(train_data, target_transform="log1p", process_as_category=True)

X_train = treated_dataset.drop(columns=["Premium Amount"])
y_train = treated_dataset["Premium Amount"]



In [4]:
X_train["Premium Amount"] = y_train


In [5]:
from utils.metrics import source_range_root_mean_squared_log_error


In [6]:
from autogluon.core.metrics import make_scorer
ag_rmsle_scorer = make_scorer(name='rmsle',
                                 score_func=source_range_root_mean_squared_log_error,
                                 optimum=0,
                                 greater_is_better=False)

In [7]:
X_train['Premium Amount']

0          7.962067
1          7.302496
2          6.342121
3          6.641182
4          7.612337
             ...   
1199995    7.173192
1199996    6.711740
1199997    5.918894
1199998    6.391917
1199999    7.816417
Name: Premium Amount, Length: 1200000, dtype: float64

In [8]:
X_train

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,...,Policy Start Year_null,Policy Start Month_null,Policy Start Day_null,Policy Start Hour_null,Policy Start Minute_null,Policy Start Second_null,Annual Income log 10_null,Previous Claims log_null,Null columns,Premium Amount
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,...,False,False,False,False,False,False,False,False,0,7.962067
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,...,False,False,False,False,False,False,False,False,1,7.302496
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,...,False,False,False,False,False,False,False,False,1,6.342121
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,...,False,False,False,False,False,False,False,False,1,6.641182
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,...,False,False,False,False,False,False,False,False,0,7.612337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,Premium,...,False,False,False,False,False,False,False,True,3,7.173192
1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,Comprehensive,...,False,False,False,False,False,False,False,True,4,6.711740
1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,Basic,...,False,False,False,False,False,False,False,False,2,5.918894
1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.547381,Suburban,Premium,...,False,False,False,False,False,False,True,False,3,6.391917


In [9]:
hyperparameter_tune_kwargs = {  
    'num_trials': 40,
    'scheduler' : 'local',
    'searcher'  : 'auto',
}

predictor = TabularPredictor(label = 'Premium Amount',
                             eval_metric = ag_rmsle_scorer,
                             problem_type = "regression",
                            )
predictor.fit(X_train,
              time_limit = 11*60*60, # 10 minutes
              hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
              presets = 'best_quality',
              save_space = True,
              keep_only_best = False,
             )

No path specified. Models will be saved in: "AutogluonModels\ag-20241227_055104"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.6
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          16
Memory Avail:       11.89 GB / 31.91 GB (37.3%)
Disk Space Avail:   119.93 GB / 931.01 GB (12.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout

(_dystack pid=37092) ╭───────────────────────────────────────────────────────────╮
(_dystack pid=37092) │ Configuration for experiment     NeuralNetFastAI_BAG_L1   │
(_dystack pid=37092) ├───────────────────────────────────────────────────────────┤
(_dystack pid=37092) │ Search algorithm                 SearchGenerator          │
(_dystack pid=37092) │ Scheduler                        FIFOScheduler            │
(_dystack pid=37092) │ Number of trials                 40                       │
(_dystack pid=37092) ╰───────────────────────────────────────────────────────────╯
(_dystack pid=37092) 
(_dystack pid=37092) View detailed results here: D:/Development/insurance/NEW_FIRST_TRY/AutogluonModels/ag-20241227_055104/ds_sub_fit/sub_fit_ho/models/NeuralNetFastAI_BAG_L1


(model_trial pid=36932) 	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 14.41% memory usage per fold, 57.63%/80.00% total).
(_ray_fit pid=21116) Metric rmsle is not supported by this model - using mean_squared_error instead
(model_trial pid=33084) 	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 17.28% memory usage per fold, 69.13%/80.00% total).
(_ray_fit pid=33676) Metric rmsle is not supported by this model - using mean_squared_error instead [repeated 4x across cluster]
(_ray_fit pid=33676) 	Ran out of time, stopping training early. (Stopping on epoch 0)
(_ray_fit pid=31944) Metric rmsle is not supported by this model - using mean_squared_error instead [repeated 3x across cluster]
(_ray_fit pid=34776) 	Ran out of time, stopping training early. (Stopping on epoch 0) [repeated 4x across cluster]
(_ray_fit pid=34280) Metric rmsle is not supported by this model - using mean_squared_error ins

(_dystack pid=37092) 


  0%|          | 0/40 [00:00<?, ?it/s]
(_dystack pid=37092) 	Memory not enough to fit 8 folds in parallel. Will train 2 folds in parallel instead (Estimated 28.14% memory usage per fold, 56.29%/80.00% total).
(_dystack pid=37092) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=8, gpus=0, memory=28.14%)
(_dystack pid=37092) 	Stopping HPO to satisfy time limit...
  0%|          | 0/40 [00:52<?, ?it/s]
(_dystack pid=37092) Fitted model: XGBoost_BAG_L1\T1 ...
(_dystack pid=37092) 	-1.0623	 = Validation score   (-rmsle)
(_dystack pid=37092) 	51.96s	 = Training   runtime
(_dystack pid=37092) 	3.19s	 = Validation runtime
(_dystack pid=37092) Hyperparameter tuning model: NeuralNetTorch_BAG_L1 ... Tuning model for up to 54.95s of the 9613.01s of remaining time.
(_ray_fit pid=27832) Metric rmsle is not supported by this model - using mean_squared_error instead [repeated 4x across cluster]


...

(_dystack pid=37092) ╭──────────────────────────────────────────────────────────────╮
(_dystack pid=37092) │ Configuration for experiment     NeuralNetTorch_r19_BAG_L1   │
(_dystack pid=37092) ├──────────────────────────────────────────────────────────────┤
(_dystack pid=37092) │ Search algorithm                 SearchGenerator             │
(_dystack pid=37092) │ Scheduler                        FIFOScheduler               │
(_dystack pid=37092) │ Number of trials                 40                          │
(_dystack pid=37092) ╰──────────────────────────────────────────────────────────────╯
(_dystack pid=37092) 
(_dystack pid=37092) View detailed results here: D:/Development/insurance/NEW_FIRST_TRY/AutogluonModels/ag-20241227_055104/ds_sub_fit/sub_fit_ho/models/NeuralNetTorch_r19_BAG_L1


(model_trial pid=23808) 	Memory not enough to fit 8 folds in parallel. Will train 1 folds in parallel instead (Estimated 45.30% memory usage per fold, 45.30%/80.00% total).
(model_trial pid=23808) 		Switching to pseudo sequential ParallelFoldFittingStrategy to avoid Python memory leakage.
(model_trial pid=23808) 		Overrule this behavior by setting fold_fitting_strategy to 'sequential_local' in ag_args_ensemble when when calling `predictor.fit`
(model_trial pid=20056) 	Memory not enough to fit 8 folds in parallel. Will train 1 folds in parallel instead (Estimated 48.31% memory usage per fold, 48.31%/80.00% total).
(model_trial pid=20056) 		Switching to pseudo sequential ParallelFoldFittingStrategy to avoid Python memory leakage.
(model_trial pid=20056) 		Overrule this behavior by setting fold_fitting_strategy to 'sequential_local' in ag_args_ensemble when when calling `predictor.fit`
(model_trial pid=20044) 	Memory not enough to fit 8 folds in parallel. Will train 1 folds in parallel in

(_dystack pid=37092) 


(_dystack pid=37092) 	No hyperparameter search space specified for XGBoost_r95_BAG_L1. Skipping HPO. Will train one model based on the provided hyperparameters.
(_dystack pid=37092) 	Memory not enough to fit 5 folds in parallel. Will train 1 folds in parallel instead (Estimated 51.27% memory usage per fold, 51.27%/80.00% total).
(_dystack pid=37092) 	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy (1 workers, per: cpus=8, gpus=0, memory=51.27%)
(_dystack pid=37092) 		Switching to pseudo sequential ParallelFoldFittingStrategy to avoid Python memory leakage.
(_dystack pid=37092) 		Overrule this behavior by setting fold_fitting_strategy to 'sequential_local' in ag_args_ensemble when when calling `predictor.fit`
(_dystack pid=37092) Fitted model: XGBoost_r95_BAG_L1 ...
(_dystack pid=37092) 	-1.0697	 = Validation score   (-rmsle)
(_dystack pid=37092) 	57.02s	 = Training   runtime
(_dystack pid=37092) 	2.1s	 = Validation runtime
(_dystack pid=37092) Hyper

(raylet) The node with node id: d4b1b7077d083d117f18e5bb041e2cfd72cf25192d86170c0833b101 and address: 127.0.0.1 and node name: 127.0.0.1 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a 	(1) raylet crashes unexpectedly (OOM, etc.) 
	(2) raylet has lagging heartbeats due to slow network or busy workload.


In [41]:
predictor

[]